# KAIST AI605 Assignment 2: Text Retrieval
TA in charge: Yongrae Jo (yongrae@kaist.ac.kr)

**Due Date:** October 27 (Wed) 11:00pm, 2021

## Your Submission
If you are a KAIST student, you will submit your assignment via [KLMS](https://klms.kaist.ac.kr). If you are a NAVER student, you will submit via [Google Form](https://forms.gle/aGZZ86YpCdv2zEVt9). 

You need to submit both (1) a PDF of this notebook, and (2) a link to CoLab for execution (.ipynb file is also allowed).

Use in-line LaTeX (see below) for mathematical expressions. Collaboration among students is allowed but it is not a group assignment so make sure your answer and code are your own. Make sure to mention your collaborators in your assignment with their names and their student ids.

## Grading
The entire assignment is out of 20 points. You can obtain up to 5 bonus points (i.e. max score is 25 points). For every late day, your grade will be deducted by 2 points (KAIST students only). You can use one of your no-penalty late days (7 days in total). Make sure to mention this in your submission. You will receive a grade of zero if you submit after 7 days.


## Environment
You will only use Python 3.7 and PyTorch 1.9, which is already available on Colab:

In [ ]:
from platform import python_version
import torch

print("python", python_version())
print("torch", torch.__version__)

python 3.7.12
torch 1.9.0+cu111


You will use two datasets, namely SQuAD, for retrieval. Note that this is a MRC dataset but we will view them as retrieval task by trying to find the correct document corresponding to the question among all the documents in the **validation** data. 

In [ ]:
!pip install -q datasets

In [ ]:
from datasets import load_dataset
from pprint import pprint

squad_dataset = load_dataset('squad')
pprint(squad_dataset['train'][0]) # 'context' contains the document

Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the '
            "Main Building's gold dome is a golden statue of the Virgin Mary. "
            'Immediately in front of the Main Building and facing it, is a '
            'copper statue of Christ with arms upraised with the legend '
            '"Venite Ad Me Omnes". Next to the Main Building is the Basilica '
            'of the Sacred Heart. Immediately behind the basilica is the '
            'Grotto, a Marian place of prayer and reflection. It is a replica '
            'of the grotto at Lourdes, France where the Virgin Mary reputedly '
            'appeared to Saint Bernadette Soubirous in 1858. At the end of the '
            'main drive (and in a direct line that connects through 3 statues '
            'and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did t

## 1. Measuring Similarity
We discussed in Lecture 08 that there are several ways to measure similarity between two vectors, such as L2 (Euclidean) distance, L1 (Manhattan) distance, inner product, and cosine distance. Here, except for inner product, all other measures are *metric* (see *Definition* at https://en.wikipedia.org/wiki/Metric_(mathematics)).

> **Problem 1.1** *(2 points)* Using the definition of metric above, prove that L1 distance is a metric.

> **Problem 1.2** *(2 points)* Prove that negative inner product is NOT a metric.

> **Problem 1.3** *(2 points)* Prove that cosine distance (1 - cosine similarity) is a metric.

> **Problem 1.4 (bonus)** *(3 points)* Given a model that can perform nearest neighbor search in L2 space, can you modify your query and your key vectors to perform maximum inner product search? (Hint: Recall the difference between MIPS and L2 NNS in Lecture 09. Can you modify key vectors so that the difference becomes 0?)

We first create an abstract class for performing similarity search as follows (`raise NotImplementedError()` means you have to override these methods when you subclass the class):

In [ ]:
class SimilaritySearch(object):
  def __init__(self):
    raise NotImplementedError()

  def train(self, documents: list):
    raise NotImplementedError()

  #Add documents (a list of text)
  def add(self, documents: list):
    raise NotImplementedError()

  #Returns the indices of top-k documents among the added documents
  #that are most similar to the input query 
  def search(self, query: str, k: int) -> list:
    raise NotImplementedError()


You will use the same space-based tokenizer that you used in Assignment 1, with lowercasing to make it case insensitive.

## 2. Sparse Search



> **Problem 2.1** *(2 points)* We will first start with Bag of Words that we discussed in Lecture 08. Using the definition in the class (don't worry about the exact definition though), implement `BagOfWords` class that subclasses `SimilaritySearch` class.

> **Problem 2.2** *(2 points)* Using the definition in Lecture 08 (don't worry about the exact definition though), implement `TFIDF` class that subclasses `BagOfWords` class.

> **Problem 2.3** *(2 points)* Use `TFIDF` to masure the recall rate of the correct document when 10 documents (contexts) are retrieved (this is called **Recall@10**) in SQuAD **validation** set.

> **Problem 2.4 (bonus)** *(2 points)* Implement `BM25` that sublcasses `BagOfWords` and repeat Problem 2.3 for BM25.

## 3. Dense Search

To obtain the embedding of each document and query, you will use GloVe embedding (Pennington et al., 2014). Go to https://nlp.stanford.edu/projects/glove/ and download `glove.6B.zip`. You are also welcome to use other more convenient ways to access the GloVe embeddings. You will compute the document's embedding by simply averaging the embeddings of all words in the document (same for the query), and then normalizing it. This way, inner product effectively becomes cosine similarity.

> **Problem 3.1** *(3 points)* Implement `GloVeAverage` class that subclasses `SimilaritySearch` class, using PyTorch's tensor native operation for the dense search. 

> **Problem 3.2** *(2 points)* Use `GloVeAverage` to measure the recall at 10 for SQuAD validation dataset. How does it compare to TFIDF?

> **Problem 3.3** *(2 points)* Implement `GloVeAverageFaiss` that subclasses `SimilaritySearch` and uses Faiss `IndexFlatIP` instead of PyTorch native tensor operation for search. Refer to the Faiss wiki (https://github.com/facebookresearch/faiss/wiki/Getting-started) for instructions.

> **Problem 3.4** *(1 points)* Compare the speed between `GloVeAverage` and `GloVeAverageFaiss` on SQuAD. To make the measurement accurate, perform search many times (at least more than 1000) and take the average.